# **QLAB - PostgreSQL para Ciencia de Datos - Avance 1**
15-FEB-2025

## **1.** Importamos las librerías necesarias

In [1]:
import pandas as pd
import psycopg2
import os

## **2.** Hacemos la carga de archivos y hacemos las transformaciones necesarias

In [2]:
# 📌 1️⃣ Cargar los archivos CSV y normalizar los datos
files = {
    "2015": "data-2015.csv",
    "2016": "data-2016.csv",
    "2017": "data-2017.csv",
    "2018": "data-2018.csv",
    "2019": "data-2019.csv",
    "migration": "net_migration_by_country.csv"
}

# Lista para almacenar los dataframes de felicidad
happiness_data = []

# Procesar cada archivo de felicidad (2015-2019)
for year, path in files.items():
    if year != "migration":  # Excluir dataset de migración
        df = pd.read_csv(path)
        
        # Renombrar columnas para estandarizarlas
        rename_dict = {
            f"Score {year}": "happiness_score",
            f"GDP {year}": "gdp_per_capita",
            f"Family {year}": "social_support",
            f"Life Expectancy {year}": "healthy_life_expectancy",
            f"Freedom {year}": "freedom_to_make_choices",
            f"Trust {year}": "corruption_perception",
            f"Generosity {year}": "generosity"
        }
        
        # Aplicar cambios de nombre y seleccionar solo las columnas necesarias
        df = df.rename(columns=rename_dict)
        df = df[["Country", "Region","happiness_score", "gdp_per_capita", "social_support",
                 "healthy_life_expectancy", "freedom_to_make_choices", "corruption_perception", "generosity"]]
        
        # Agregar la columna del año
        df["year"] = int(year)
        
        # Agregar a la lista de dataframes
        happiness_data.append(df)

# Unir todos los datasets de felicidad en uno solo
happiness_df = pd.concat(happiness_data, ignore_index=True)

# Renombrar columna "Country" a "country" para que coincida con el dataset de migración
happiness_df = happiness_df.rename(columns={"Country": "country"})
happiness_df = happiness_df.rename(columns={"Region": "region"})


# 📌 2️⃣ Cargar y filtrar los datos de migración
migration_df = pd.read_csv(files["migration"])
migration_df = migration_df[(migration_df["year"] >= 2015) & (migration_df["year"] <= 2019)]

# Renombrar columnas para que coincidan con happiness_df
migration_df = migration_df.rename(columns={
    "country_name": "country",
    "value": "net_migration"
})

# Seleccionar solo las columnas necesarias
migration_df = migration_df[["country", "year", "net_migration"]]

## **3.** Conexión a postgre, creación de BBDD y carga de tablas 
Recordar cambiar las credenciales necesarias para hacer la conexión con PostgreSQL y tener este script en la misma carpeta que los dataset (.csv).

In [3]:
# 📌 3️⃣ Conectar a PostgreSQL y crear la base de datos "avance_1" si no existe
def crear_base_de_datos():
    try:
        conn = psycopg2.connect(
            dbname="postgres",
            user="postgres",
            password="485174",
            host="localhost",
            port="7777"
        )
        conn.autocommit = True
        cursor = conn.cursor()

        # Crear la base de datos si no existe
        cursor.execute("SELECT 1 FROM pg_database WHERE datname = 'avance_1';")
        existe = cursor.fetchone()

        if not existe:
            cursor.execute("CREATE DATABASE avance_1;")
            print("✅ Base de datos 'avance_1' creada correctamente.")
        else:
            print("⚠️ La base de datos 'avance_1' ya existe.")

        cursor.close()
        conn.close()
    except Exception as e:
        print(f"❌ Error al crear la base de datos: {e}")

crear_base_de_datos()

# 📌 4️⃣ Conectar a la nueva base de datos "avance_1"
def conectar_a_base_de_datos():
    try:
        conn = psycopg2.connect(
            dbname="avance_1",
            user="postgres",
            password="485174",
            host="localhost",
            port="7777"
        )
        print("✅ Conexión exitosa a la base de datos 'avance_1'.")
        return conn
    except Exception as e:
        print(f"❌ Error al conectar a la base de datos: {e}")
        return None

conn = conectar_a_base_de_datos()
cursor = conn.cursor()

# 📌 5️⃣ Crear las tablas en PostgreSQL
cursor.execute("""
    CREATE TABLE IF NOT EXISTS Country (
    id SERIAL PRIMARY KEY,
    country VARCHAR(25) UNIQUE,
    region VARCHAR(32) UNIQUE
    )
""")

cursor.execute("""
    CREATE TABLE IF NOT EXISTS Happiness (
    id SERIAL PRIMARY KEY,
    country_id INT REFERENCES Country(id),
    year INT,
    happiness_score FLOAT,
    gdp_per_capita FLOAT,
    social_support FLOAT,
    healthy_life_expectancy FLOAT,
    freedom_to_make_choices FLOAT,
    corruption_perception FLOAT,
    generosity FLOAT
    )
""")

cursor.execute("""
    CREATE TABLE IF NOT EXISTS Migration (
    id SERIAL PRIMARY KEY,
    country_id INT REFERENCES Country(id),
    year INT,
    net_migration INT
    )
""")
conn.commit()
print("✅ Tablas 'Country', 'Happiness' y 'Migration' creadas correctamente en 'avance_1'.")

# 📌 6️⃣ Insertar datos en la tabla Country
print("⏳ Insertando datos en Country...")
cursor.execute("""
        INSERT INTO Country (country, region)
        VALUES (%s, %s)
    """, (row["country"][:25], row["region"][:25]))  # Recorta a 25 caracteres

print("✅ Datos insertados en Country.")

# 📌 7️⃣ Insertar datos en la tabla Happiness
print("⏳ Insertando datos en Happiness...")
for _, row in happiness_df.iterrows():
    cursor.execute("""
        INSERT INTO Happiness (year, happiness_score, gdp_per_capita, social_support, 
                                      healthy_life_expectancy, freedom_to_make_choices, corruption_perception, 
                                      generosity)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """, (row["year"], row["happiness_score"], row["gdp_per_capita"], 
          row["social_support"], row["healthy_life_expectancy"], row["freedom_to_make_choices"],
          row["corruption_perception"], row["generosity"]))

print("✅ Datos insertados en migration_data.")

# 📌 8️⃣ Insertar datos en la tabla Migration
print("⏳ Insertando datos en Migration...")
for _, row in migration_df.iterrows():
    cursor.execute("""
        INSERT INTO Migration (year, net_migration)
        VALUES (%s, %s)
    """, (row["year"], row["net_migration"]))

print("✅ Datos insertados en Migration.")

# 📌 8️⃣ Confirmar y cerrar conexión
conn.commit()
cursor.close()
conn.close()
print("✅ Conexión cerrada.")

⚠️ La base de datos 'avance_1' ya existe.
✅ Conexión exitosa a la base de datos 'avance_1'.
✅ Tablas 'Country', 'Happiness' y 'Migration' creadas correctamente en 'avance_1'.
⏳ Insertando datos en Country...


NameError: name 'row' is not defined

In [28]:
print(happiness_df["country"].apply(len).max())  # Máxima longitud de 'country'
print(happiness_df["region"].apply(len).max())   # Máxima longitud de 'region'

24
31
